In [124]:
#%% Importing modules and data
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import os
import quandl as qd
import seaborn as sns
import matplotlib.dates as dates
import matplotlib.ticker as ticker
from pandas_datareader import data
from lxml import html
import requests
import webbrowser
from bs4 import BeautifulSoup as bs
import json
import csv
import os
import glob
from time import sleep

In [35]:
# Importing cross currency sheet
cryptos = pd.read_csv('crosspairs.csv')

'''
for idx, row in cryptos.iterrows():
    symbol = row['Symbol']
    exchanges = row.dropna().index.values[2:]
    
exchanges
'''

array(['Cryptopia', 'Binance', 'Bittrex'], dtype=object)

In [188]:
# Pull from Cryptopia
def pulltopia(symbol,base):
    # Fields:
    # AskPrice BaseVolume BidPrice BuyBaseVolume BuyVolume Change
    # Close High Label LastPrice Low Open SellBaseVolume 
    # SellVolume TradePairId Volume
    
    topiasite = 'https://www.cryptopia.co.nz/api/GetMarket/' + symbol + '_' + base + '/0.001'
    response = requests.get(topiasite)
    json_data = json.loads(response.text)
    return pd.DataFrame(json_data['Data'], index = [0])

# Pull from kucoin
def pullkucoin(symbol,base):
    kubuy = 'https://api.kucoin.com/v1/' + symbol + '-' + base + '/open/orders-buy/'
    kusell = 'https://api.kucoin.com/v1/' + symbol + '-' + base + '/open/orders-sell/'
    respone = requests.get(kubuy)
    json_data_buy = json.loads(respone.text)
    response = requests.get(kusell)
    json_data_sell = json.loads(response.text)
    return pd.DataFrame({'BidPrice':[x[0] for x in json_data_buy['data']],
                         'BuyBaseVolume': [x[1] for x in json_data_buy['data']],
                         'AskPrice': [x[0] for x in json_data_sell['data']],
                         'SellBaseVolume': [x[1] for x in json_data_sell['data']]},
                        index = range(len(json_data_buy['data'])))

# Pull from Bittrex
def trexpull(symbol,base):
    trexsite = 'https://bittrex.com/api/v1.1/public/getorderbook?market=' + base + '-' + symbol + '&type=both'
    response = requests.get(trexsite)
    json_data = json.loads(response.text)
    return pd.concat([pd.DataFrame({'BidPrice': [x['Rate'] for x in json_data['result']['buy']],
                                    'BuyBaseVolume': [x['Quantity'] for x in json_data['result']['buy']]},
                                   index = range(len(json_data['result']['buy']))),
                      pd.DataFrame({'AskPrice': [x['Rate'] for x in json_data['result']['sell']],
                                    'SellBaseVolume': [x['Quantity'] for x in json_data['result']['sell']]},
                                   index = range(len(json_data['result']['sell'])))],
                     axis = 1)

# Pull from Quoine
def quoinepull(id):
    # currencypair id
    # UBTCETH-->75
    # UBTCBTC-->74
    # BTCUSD-->1
    # QASHBTC-->52
    # ETHBTC-->37
    # ETHUSD-->27
    # NEOUSD-->53
    # 
    quoinesite = 'https://api.quoine.com/products/' + str(id) + '/price_levels'
    response = requests.get(quoinesite)
    json_data = json.loads(response.text)
    return pd.DataFrame({'BidPrice': [x[0] for x in json_data['buy_price_levels']],
                         'BuyBaseVolume': [x[1] for x in json_data['buy_price_levels']],
                         'AskPrice': [x[0] for x in json_data['sell_price_levels']],
                         'SellBaseVolume': [x[1] for x in json_data['sell_price_levels']]},
                        index = range(len(json_data['buy_price_levels'])))

In [233]:
# Arbitrage from Kucoin and Trex

def kutrexarb(symbol,base):
    ku = pullkucoin(symbol,base)
    ku.columns = ['ku_' + colname for colname in ku]
    trex = trexpull(symbol,base)[:6]
    trex.columns = ['trex_' + colname for colname in trex]
    return pd.concat([ku,trex],axis = 1)

def checkarbs():
    pairs = pd.read_csv('kutrex.csv')
    currencypairs = []
    buyex = []
    sellex = []
    askprice = []
    bidprice = []
    arbamount = []
    for idx, row in pairs.iterrows():
        symbol = row['Symbol']
        base = row['Base']
        tempseries = kutrexarb(symbol,base).iloc[0]
        
        if tempseries['ku_AskPrice'] < tempseries['trex_BidPrice']:
            # Buy on Ku and sell on Trex
            currencypairs = currencypairs + [symbol + base]
            buyex = buyex + ['Ku']
            sellex = sellex + ['Trex']
            askprice = askprice + [tempseries['ku_AskPrice']]
            bidprice = bidprice + [tempseries['trex_BidPrice']]
            arbamount = arbamount + [tempseries['trex_BidPrice']/tempseries['ku_AskPrice']-1]
        elif tempseries['trex_AskPrice'] < tempseries['ku_BidPrice']:
            # Buy on Trex ad sell on Ku
            currencypairs = currencypairs + [symbol + base]
            buyex = buyex + ['Trex']
            sellex = sellex + ['Ku']
            askprice = askprice + [tempseries['trex_AskPrice']]
            bidprice = bidprice + [tempseries['ku_BidPrice']]
            arbamount = arbamount + [tempseries['ku_BidPrice']/tempseries['trex_AskPrice']-1]
        else:
            continue
        sleep(0.1)
    
    df = pd.DataFrame({'BuyExchange': buyex,
                      'SellExchange': sellex,
                      'AskPrice': askprice,
                      'BidPrice': bidprice,
                      'ArbReturn': arbamount},
                     index = currencypairs)
    return df.sort_values(['ArbReturn'])

In [237]:
checkarbs()

,ArbReturn,AskPrice,BidPrice,BuyExchange,SellExchange
LTCBTC,0.000525,0.017071,0.017080,Trex,Ku
BTCUSDT,0.000667,13660.898329,13670.010000,Ku,Trex
SNTBTC,0.001026,0.000029,0.000029,Ku,Trex
DASHBTC,0.001179,0.075300,0.075389,Ku,Trex
NEOBTC,0.003199,0.008365,0.008392,Trex,Ku
DASHETH,0.003727,0.847000,0.850157,Ku,Trex
LTCETH,0.006349,0.191102,0.192315,Ku,Trex
OMGBTC,0.016235,0.001624,0.001650,Trex,Ku
BTGETH,0.124911,0.181199,0.203833,Trex,Ku
BTGBTC,0.127785,0.015800,0.017819,Trex,Ku


In [238]:
kutrexarb('BTG','ETH')

,ku_AskPrice,ku_BidPrice,ku_BuyBaseVolume,ku_SellBaseVolume,trex_BidPrice,trex_BuyBaseVolume,trex_AskPrice,trex_SellBaseVolume
0,0.847000,0.831800,0.120221,0.000237,0.850157,0.013655,0.871393,18.088900
1,0.872000,0.831799,2.138975,0.565732,0.850156,0.015893,0.871393,46.980148
2,0.899949,0.830960,0.098402,0.092084,0.845855,0.013655,0.871393,4.777831
3,0.899990,0.808000,0.200000,0.106036,0.845855,0.016383,0.871393,34.835000
4,0.899998,0.780000,2.000000,0.015718,0.841574,27.055400,0.871393,36.908000
5,0.900000,0.760000,0.063117,1.000000,0.841574,45.200000,0.871393,33.817000
